# Introduction to XGBoost Spark with GPU

Agaricus is an example of xgboost classifier for multiple classification. This notebook will show you how to load data, train the xgboost model.

A few libraries required for this notebook:
  1. NumPy
  2. cudf jar
  3. xgboost4j jar
  4. xgboost4j-spark jar
  5. rapids-4-spark.jar
  
This notebook also illustrates the ease of porting a sample CPU based Spark xgboost4j code into GPU. There is only one change required for running Spark XGBoost on GPU. That is replacing the API `setFeaturesCol(feature)` on CPU with the new API `setFeaturesCols(features)`. This also eliminates the need for vectorization (assembling multiple feature columns in to one column) since we can read multiple columns.

In [1]:
!nvidia-smi

Fri Apr  8 14:40:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   28C    P0    22W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip3 install -q findspark

In [3]:
import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/opt/spark-3.2.1-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--files /opt/spark-3.2.1-bin-hadoop3.2/examples/src/main/scripts/getGpusResources.sh --conf spark.plugins=com.nvidia.spark.SQLPlugin --jars /opt/xgboost/cudf-22.02.0-cuda11.jar,/opt/xgboost/rapids-4-spark_2.12-22.02.0.jar,/opt/xgboost/xgboost4j_3.0-1.4.2-0.2.0.jar,/opt/xgboost/xgboost4j-spark_3.0-1.4.2-0.2.0.jar pyspark-shell'


import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.sparkContext.addPyFile('/opt/xgboost/xgboost4j-spark_3.0-1.4.2-0.2.0.jar')

/opt/spark-3.2.1-bin-hadoop3.2/python/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [4]:
#JAR PATHS
#/opt/xgboost/cudf-22.02.0-cuda11.jar
#/opt/xgboost/rapids-4-spark_2.12-22.02.0.jar
#/opt/xgboost/xgboost4j_3.0-1.4.2-0.1.0.jar
#/opt/xgboost/xgboost4j-spark_3.0-1.4.2-0.1.0.jar

#### Import All Libraries

In [5]:
from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel, XGBoostClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType, StructField, StructType
from time import time
import os

Besides CPU version requires two extra libraries.
```Python
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
```

#### Create Spark Session and Data Reader

In [6]:
# Make sure it runs on GPU
spark.conf.set("spark.rapids.sql.enabled", "true")
spark.conf.set("spark.rapids.sql.incompatibleDateFormats.enabled","true")
spark.conf.set("spark.rapids.sql.csv.read.integer.enabled","true")
spark.conf.set("spark.rapids.sql.csv.read.long.enabled","true")
spark.conf.set("spark.rapids.sql.csv.read.double.enabled","true")
spark.conf.set("spark.rapids.sql.incompatibleDateFormats.enabled","true")

#spark.conf.set("spark.plugins=com.nvidia.spark.SQLPlugin")
spark.conf.set("spark.rapids.memory.gpu.pooling.enabled","false")
spark.conf.set("spark.executor.resource.gpu.amount","1")
spark.conf.set("spark.task.resource.gpu.amount","1")
spark.conf.set("spark.executor.resource.gpu.discoveryScript","/opt/spark-3.2.1-bin-hadoop3.2/examples/src/main/scripts/getGpusResources.sh")

reader = spark.read

#### Specify the Data Schema and Load the Data

In [7]:
label = 'label'
features = [ 'feature_' + str(i) for i in range(0, 126) ]
schema = StructType([ StructField(x, FloatType()) for x in [label] + features ])

# You need to update them to your real paths!
dataRoot = os.getenv("DATA_ROOT", "/opt/xgboost")
train_data = reader.schema(schema).option('header', True).csv(dataRoot + '/agaricus/csv/train')
trans_data = reader.schema(schema).option('header', True).csv(dataRoot + '/agaricus/csv/test')

Note on CPU version, vectorization is required before fitting data to classifier, which means you need to assemble all feature columns into one column.

```Python
def vectorize(data_frame):
    to_floats = [ col(x.name).cast(FloatType()) for x in data_frame.schema ]
    return (VectorAssembler()
        .setInputCols(features)
        .setOutputCol('features')
        .transform(data_frame.select(to_floats))
        .select(col('features'), col(label)))

train_data = vectorize(train_data)
trans_data = vectorize(trans_data)
```

#### Create a XGBoostClassifier

In [8]:
params = { 
    'eta': 0.1,
    'missing': 0.0,
    'treeMethod': 'gpu_hist',
    'maxDepth': 2,
    'numWorkers': 1,
    'numRound' : 100,
}
classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCols(features)

The CPU version classifier provides the API `setFeaturesCol` which only accepts a single column name, so vectorization for multiple feature columns is required.
```Python
classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCol('features')
```

The parameter `num_workers` should be set to the number of GPUs in Spark cluster for GPU version, while for CPU version it is usually equal to the number of the CPU cores.

Concerning the tree method, GPU version only supports `gpu_hist` currently, while `hist` is designed and used here for CPU training.

#### Train the Data with Benchmark

In [9]:
def with_benchmark(phrase, action):
    start = time()
    result = action()
    end = time()
    print('{} takes {} seconds'.format(phrase, round(end - start, 2)))
    return result
model = with_benchmark('Training', lambda: classifier.fit(train_data))

Training takes 17.85 seconds


#### Transformation and Show Result Sample

In [10]:
def transform():
    result = model.transform(trans_data).cache()
    result.foreachPartition(lambda _: None)
    return result
result = with_benchmark('Transformation', transform)
result.select(label, 'rawPrediction', 'probability', 'prediction').show(5)

Transformation takes 4.98 seconds
+-----+--------------------+--------------------+----------+
|label|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+----------+
|  1.0|[-0.9997572302818...|[2.42769718170166...|       1.0|
|  0.0|[-0.9960258603096...|[0.00397413969039...|       1.0|
|  0.0|[-0.9974485635757...|[0.00255143642425...|       1.0|
|  0.0|[-0.0711352527141...|[0.92886474728584...|       0.0|
|  0.0|[-0.9983345270156...|[0.00166547298431...|       1.0|
+-----+--------------------+--------------------+----------+
only showing top 5 rows



#### Evaluation

In [11]:
accuracy = with_benchmark(
    'Evaluation',
    lambda: MulticlassClassificationEvaluator().setLabelCol(label).evaluate(result))
print('Accuracy is ' + str(accuracy))

Evaluation takes 0.32 seconds
Accuracy is 0.9069677632722861


#### Stop

In [12]:
spark.stop()